In [1]:
# Install this if yfinacne is not installed.
# %pip install yfinance --upgrade --no-cache-dir

In [2]:
# Load packages
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Read csv as a Pandas DataFrame
# https://www.kaggle.com/datasets/lsind18/euro-exchange-daily-rates-19992020/
ex_rates = pd.read_csv('euro-daily-hist_1999_2022.csv')

In [4]:
# See the header names in this dataset
list(ex_rates.columns)

['Period\\Unit:',
 '[Australian dollar ]',
 '[Bulgarian lev ]',
 '[Brazilian real ]',
 '[Canadian dollar ]',
 '[Swiss franc ]',
 '[Chinese yuan renminbi ]',
 '[Cypriot pound ]',
 '[Czech koruna ]',
 '[Danish krone ]',
 '[Estonian kroon ]',
 '[UK pound sterling ]',
 '[Greek drachma ]',
 '[Hong Kong dollar ]',
 '[Croatian kuna ]',
 '[Hungarian forint ]',
 '[Indonesian rupiah ]',
 '[Israeli shekel ]',
 '[Indian rupee ]',
 '[Iceland krona ]',
 '[Japanese yen ]',
 '[Korean won ]',
 '[Lithuanian litas ]',
 '[Latvian lats ]',
 '[Maltese lira ]',
 '[Mexican peso ]',
 '[Malaysian ringgit ]',
 '[Norwegian krone ]',
 '[New Zealand dollar ]',
 '[Philippine peso ]',
 '[Polish zloty ]',
 '[Romanian leu ]',
 '[Russian rouble ]',
 '[Swedish krona ]',
 '[Singapore dollar ]',
 '[Slovenian tolar ]',
 '[Slovak koruna ]',
 '[Thai baht ]',
 '[Turkish lira ]',
 '[US dollar ]',
 '[South African rand ]']

In [5]:
# Extract a subset as we only want excahnge rates between USD and EUR
USDEUR = ex_rates[['Period\\Unit:', '[US dollar ]']]
# Rename the columns
USDEUR = USDEUR.rename(columns = {'Period\\Unit:':'Date', '[US dollar ]':'USDEUR'})

In [6]:
# Convert date data to the correct/appropriate date format
USDEUR['Date'] = pd.to_datetime(USDEUR['Date'], infer_datetime_format = True)
# Set Date as the index/row headers
USDEUR = USDEUR.set_index('Date')

In [7]:
# With the date format set, we can take a subset of the data for only the relevent dates
# And covert the exchange rates to float
USDEUR = USDEUR.loc['2021-12-31':'2021-01-01'].astype('float64')

In [8]:
# yfinance is an API package which downloads historical finance data
# It saves as a Pandas DataFrame automatically
df = yf.download(
    tickers = "AAPL BTC-USD",
    start="2021-01-01", end="2021-12-31",
    interval = "1d",
    auto_adjust = True,
    prepost = True,
    threads = True,
    proxy = None
    )

df = df[["Open"]]
df.columns = ['AAPL', 'BTCUSD']

[*********************100%***********************]  2 of 2 completed


In [9]:
# A few ways to merge these datasets
master_USD = pd.concat([USDEUR, df], axis=1)
# Merge keeps data with dates that exist in both tables only, so we have less record
master_USD = pd.merge(USDEUR, df, left_index=True, right_index=True)
# Remove NAs, can use either lines above and will result in the same results after dropna()
master_USD = master_USD.dropna()

In [10]:
# Convert the USD prices into EUR
master_EUR = pd.concat([pd.DataFrame({'EUR':master_USD.AAPL / master_USD.USDEUR, 'Ticker':"AAPL"}),
                        pd.DataFrame({'EUR':master_USD.BTCUSD / master_USD.USDEUR, 'Ticker':"BTCEUR"})])

In [11]:
# If you wanna see the data by date
master_EUR.sort_index(ascending = True)

,EUR,Ticker
Date,,
2021-01-04,107.627374,AAPL
2021-01-04,26684.246274,BTCEUR
2021-01-05,26059.034321,BTCEUR
2021-01-05,104.106925,AAPL
2021-01-06,102.601673,AAPL
...,...,...
2021-12-28,44726.731423,BTCEUR
2021-12-29,158.222640,AAPL
2021-12-29,42133.832694,BTCEUR


In [12]:
# If you wanna see the data by ticker and date
master_EUR.sort_values(by = ['Ticker', 'Date'], ascending = [True, True])

,EUR,Ticker
Date,,
2021-01-04,107.627374,AAPL
2021-01-05,104.106925,AAPL
2021-01-06,102.601673,AAPL
2021-01-07,103.636580,AAPL
2021-01-08,107.149604,AAPL
...,...,...
2021-12-23,42994.114721,BTCEUR
2021-12-27,44910.368967,BTCEUR
2021-12-28,44726.731423,BTCEUR


In [13]:
master_EUR = master_EUR.sort_values(by = ['Ticker', 'Date'], ascending = [True, True])

In [14]:
# Add a column to the DataFrame to record the month of dates
master_EUR['Month'] = pd.DatetimeIndex(master_EUR.index).month

In [15]:
# Count how many entries per ticker
master_EUR.groupby(['Ticker']).count()

,EUR,Month
Ticker,,
AAPL,250,250
BTCEUR,250,250


In [16]:
# Use loops to extract the data as lists
AAPL_EUR = []
AAPL_EUR_Date = []
for lab, row in master_EUR.iterrows():
    AAPL_EUR.append(row["EUR"])
    AAPL_EUR_Date.append(lab.to_pydatetime().strftime("%d/%m/%Y"))

BTCEUR = []
BTCEUR_Date = []
for lab, row in master_EUR.iterrows():
    BTCEUR.append(row["EUR"])
    BTCEUR_Date.append(lab.to_pydatetime().strftime("%d/%m/%Y"))

In [17]:
# Can convert these lists into Dictionaries
AAPL_EUR_Dict = dict(zip(AAPL_EUR_Date, AAPL_EUR))
BTCEUR_Dict = dict(zip(BTCEUR_Date, BTCEUR))

In [18]:
# Alternatively, we can turn these into numoy arrays, better for data analysis (apart from DataFrame itself)

# Create emoty arrays for storage
AAPL_EUR = np.zeros(shape=(master_EUR.groupby(['Ticker']).count().loc[['AAPL'],['EUR']].squeeze(), 2))
BTCEUR = np.zeros(shape=(master_EUR.groupby(['Ticker']).count().loc[['BTCEUR'],['EUR']].squeeze(), 2))

# Extraction
# We can use i, j for loops instead but I want to use iterrows() as covered in this course
i = 0
j = 0
for lab, row in master_EUR.iterrows():
    if row["Ticker"] == "AAPL":
        AAPL_EUR[i,0] = lab.to_numpy()
        AAPL_EUR[i,1] = row["EUR"]
        i = i + 1
    else:
        BTCEUR[j,0] = lab.to_numpy()
        BTCEUR[j,1] = row["EUR"]
        j = j + 1

In [19]:
# We can use numpy function `round()` to simplify the data.
np.round(AAPL_EUR, 0)
np.round(BTCEUR, 0)

array([[1.6097184e+18, 2.6684000e+04],
       [1.6098048e+18, 2.6059000e+04],
       [1.6098912e+18, 2.7568000e+04],
       [1.6099776e+18, 3.0005000e+04],
       [1.6100640e+18, 3.2148000e+04],
       [1.6103232e+18, 3.1527000e+04],
       [1.6104096e+18, 2.9205000e+04],
       [1.6104960e+18, 2.7877000e+04],
       [1.6105824e+18, 3.0786000e+04],
       [1.6106688e+18, 3.2300000e+04],
       [1.6110144e+18, 3.0203000e+04],
       [1.6111008e+18, 2.9791000e+04],
       [1.6111872e+18, 2.9240000e+04],
       [1.6112736e+18, 2.5348000e+04],
       [1.6115328e+18, 2.6568000e+04],
       [1.6116192e+18, 2.6648000e+04],
       [1.6117056e+18, 2.6881000e+04],
       [1.6117920e+18, 2.5177000e+04],
       [1.6118784e+18, 2.8278000e+04],
       [1.6121376e+18, 2.7404000e+04],
       [1.6122240e+18, 2.7842000e+04],
       [1.6123104e+18, 2.9550000e+04],
       [1.6123968e+18, 3.1240000e+04],
       [1.6124832e+18, 3.0820000e+04],
       [1.6127424e+18, 3.2338000e+04],
       [1.6128288e+18, 3.

In [38]:
# Correlation Matrix between AAPL and BTC
np.corrcoef(AAPL_EUR[:,1], BTCEUR[:,1])

array([[1.        , 0.28264457],
       [0.28264457, 1.        ]])

In [39]:
# Correlation between AAPL and BTC, round to 5 decimal places
np.round(np.corrcoef(AAPL_EUR[:,1], BTCEUR[:,1])[0,1], 5)

0.28264

In [20]:
# Aggregation on the data
master_EUR_agg = master_EUR.groupby(['Ticker', 'Month']).agg(['min','mean','max','std'])
# Rename headers
master_EUR_agg.columns = ['Min','Mean','Max','STD']
master_EUR_agg

Min          Mean           Max          STD
Ticker Month                                                       
AAPL   1        102.601673    108.355590    117.352591     4.863163
       2        100.393446    108.318874    113.776323     4.450951
       3         99.338425    101.845350    105.972755     1.580493
       4        104.502828    109.419984    111.957489     2.131128
       5        100.278709    103.994002    108.823702     2.439540
       6        101.406522    106.914632    113.931707     4.141694
       7        114.291494    121.623481    125.660270     3.121361
       8        121.987131    125.040625    128.460540     1.724993
       9        121.737522    126.091503    132.173914     3.306741
       10       119.725388    124.432103    128.691549     2.894300
       11       127.585077    134.118902    143.057227     5.633117
       12       139.611456    152.579424    160.156181     5.275295
BTCEUR 1      25176.611542  28541.751013  32299.519122  2252.214944
       2      27403.656178  37369.969103  47418.394693  5846.374951
       3      37467.438734  45570.316193  50260.364472  3483.738536
       4      40610.503077  47649.499280  53095.748835  3459.233847
       5      28414.971570  39193.787158  47868.248084  7100.530442
       6      26586.831136  30067.227599  33388.807374  1955.472334
       7      25311.149470  28887.244750  33686.055020  2372.364794
       8      32217.630918  38277.689661  42216.650474  3040.407520
       9      34681.518565  38775.922137  44401.754190  2391.195122
       10     37773.053610  49427.881755  56717.568424  4837.984655
       11     50008.486267  53796.751530  58348.220070  2556.523515
       12     41018.709607  44150.304540  50460.685328  2860.146245

In [21]:
# Preparation for visualisation, duplicating the (multi-) index columns for simplicity
for lab, row in master_EUR_agg.iterrows():
    master_EUR_agg.loc[lab, "Ticker"] = lab[0]
    master_EUR_agg.loc[lab, "Month"] = calendar.month_name[lab[1]]
master_EUR_agg

Min          Mean           Max          STD  Ticker  \
Ticker Month                                                                  
AAPL   1        102.601673    108.355590    117.352591     4.863163    AAPL   
       2        100.393446    108.318874    113.776323     4.450951    AAPL   
       3         99.338425    101.845350    105.972755     1.580493    AAPL   
       4        104.502828    109.419984    111.957489     2.131128    AAPL   
       5        100.278709    103.994002    108.823702     2.439540    AAPL   
       6        101.406522    106.914632    113.931707     4.141694    AAPL   
       7        114.291494    121.623481    125.660270     3.121361    AAPL   
       8        121.987131    125.040625    128.460540     1.724993    AAPL   
       9        121.737522    126.091503    132.173914     3.306741    AAPL   
       10       119.725388    124.432103    128.691549     2.894300    AAPL   
       11       127.585077    134.118902    143.057227     5.633117    AAPL   
       12       139.611456    152.579424    160.156181     5.275295    AAPL   
BTCEUR 1      25176.611542  28541.751013  32299.519122  2252.214944  BTCEUR   
       2      27403.656178  37369.969103  47418.394693  5846.374951  BTCEUR   
       3      37467.438734  45570.316193  50260.364472  3483.738536  BTCEUR   
       4      40610.503077  47649.499280  53095.748835  3459.233847  BTCEUR   
       5      28414.971570  39193.787158  47868.248084  7100.530442  BTCEUR   
       6      26586.831136  30067.227599  33388.807374  1955.472334  BTCEUR   
       7      25311.149470  28887.244750  33686.055020  2372.364794  BTCEUR   
       8      32217.630918  38277.689661  42216.650474  3040.407520  BTCEUR   
       9      34681.518565  38775.922137  44401.754190  2391.195122  BTCEUR   
       10     37773.053610  49427.881755  56717.568424  4837.984655  BTCEUR   
       11     50008.486267  53796.751530  58348.220070  2556.523515  BTCEUR   
       12     41018.709607  44150.304540  50460.685328  2860.146245  BTCEUR   

                  Month  
Ticker Month             
AAPL   1        January  
       2       February  
       3          March  
       4          April  
       5            May  
       6           June  
       7           July  
       8         August  
       9      September  
       10       October  
       11      November  
       12      December  
BTCEUR 1        January  
       2       February  
       3          March  
       4          April  
       5            May  
       6           June  
       7           July  
       8         August  
       9      September  
       10       October  
       11      November  
       12      December

In [40]:
# We can filter out and see the AAPL data
master_EUR_agg[master_EUR_agg['Ticker'] == "AAPL"]

,,Min,Mean,Max,STD,Ticker,Month
Ticker,Month,,,,,,
AAPL,7,114.291494,121.623481,125.66027,3.121361,AAPL,July
BTCEUR,7,25311.149470,28887.244750,33686.05502,2372.364794,BTCEUR,July


In [41]:
# Or the July data...
master_EUR_agg[master_EUR_agg['Month'] == "July"]

,,Min,Mean,Max,STD,Ticker,Month
Ticker,Month,,,,,,
AAPL,7,114.291494,121.623481,125.66027,3.121361,AAPL,July
BTCEUR,7,25311.149470,28887.244750,33686.05502,2372.364794,BTCEUR,July


In [52]:
# Scatter-plot of price standard deviation against mean
# Data not meaningful as BTC mean and STD values are much bigger than AAPL
fig1 = px.scatter(master_EUR_agg,
                  x = 'Mean',
                  y = 'STD',
                  color = "Ticker",
                  hover_data = ['Month'],
                  title = "Mean Price vs Standard Deviation",
                  labels = {
                     "Mean" : "Mean Price in EUR",
                     "STD" : "Standard Deviation"
                 })
fig1.show()

In [53]:
# Same plot but just with AAPL figures, scales on axises are much smaller
fig2 = px.scatter(master_EUR_agg[master_EUR_agg['Ticker'] == "AAPL"],
                 x = 'Mean',
                 y = 'STD',
                 color = "STD",
                 hover_data = ['Month'],
                 title = "AAPL",
                 labels = {
                     "Mean" : "Mean Price in EUR for AAPL",
                     "STD" : "Standard Deviation"
                 })
fig2.show()

In [54]:
# Line plot of EUR prices
# Again, we can't really see AAPL prices...
fig3 = px.line(master_EUR,
              x = master_EUR.index,
              y = 'EUR',
              color = "Ticker",
              title = "EUR per unit")
fig3.show()

In [55]:
# Same plot but just with AAPL figures, scales on axises are much smaller
fig4 = px.line(master_EUR[master_EUR['Ticker'] == "AAPL"],
              x = master_EUR[master_EUR['Ticker'] == "AAPL"].index,
              y = 'EUR',
              title = "AAPL per unit",
              labels = {
                     "x": "Date"
                 })
fig4.show()

In [56]:
# Bar plot of AAPL mean price each month
fig5 = px.bar(master_EUR_agg[master_EUR_agg['Ticker'] == "AAPL"],
              x = 'Month',
              y = 'Mean',
              title = "AAPL",
              labels = {
                     "Mean" : "Mean Price in EUR for AAPL"
                 })
fig5.show()

In [57]:
# Bar plot of BTC mean price each month
fig6 = px.bar(master_EUR_agg[master_EUR_agg['Ticker'] == "BTCEUR"],
              x = 'Month',
              y = 'Mean',
              title = "BTC",
              labels = {
                     "Mean" : "Mean Price in EUR for BTC"
                 })
fig6.show()

In [28]:
# Download same data again but just with AAPL
df2 = yf.download(
        tickers = "AAPL",
        start = "2021-01-01", end = "2021-12-31",
        interval = "1d",
        auto_adjust = True,
        prepost = True,
        threads = True,
        proxy = None
        )

[*********************100%***********************]  1 of 1 completed


In [59]:
# Interactive Candlestick plot for AAPL
fig7 = go.Figure(data = [go.Candlestick(x = df2.index,
                open = df2['Open'],
                high = df2['High'],
                low = df2['Low'],
                close = df2['Close'])]
                )

fig7.show()

# Project Report

[GitHub URL](https://github.com/marcohoucheng/UCDPA_MarcoLam)

## Abstract
(Short overview of the entire project and features)

## Introduction
(Explain why you chose this project use case)

## Dataset

There are 2 data sources used in this report. The first is a hostorical exchange rates dataset downloaded directly from [Kaggle](https://www.kaggle.com/datasets/lsind18/euro-exchange-daily-rates-19992020/) as a `.csv` file. According to the `About` session on the website, these rates are usually updated around 16:00 CET on every working day.

The second data source is an API package [`yfinance`](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiz26v_p4P5AhWjMewKHSGaDBoQFnoECAQQAQ&url=https%3A%2F%2Fpypi.org%2Fproject%2Fyfinance%2F&usg=AOvVaw0_AxElbwZGER2NXe5JdGpa), through this package I will be able to download historical prices for investment of interest. The API typically returns the low, high, open and close prices. In this report, I will use this tool to get historical prices for Apple Inc. (AAPL) for the year 2021.

Although hostorical exchange rates can be captured through packages such as `yfinance`, I have chosen Kaggle in this report to show my knowledge in importing data through various methods.

## Implementation Process

The AAPL data downloaded from `yfinance` are in US dollars, we will need to use the Kaggle dataset to convert these prices to Euro. As the exchange rates are usually updated around 16:00 CET on every working day, we will mainly focus on the opening prices for AAPL as it is consistently closest to 16:00 CET on each working day.

To convert currencies, we must first ensure that our data from different sources have the same data type and structure. The `yfinance` output is `DataFrame` from `Pandas`, so it would make sense to use the `read_csv()` function from `Pandas` to load in the `.csv` file. After inspecting this dataset, we need to change the date format to `DateTime` and make it the index of the dataset. We can also take a subset of the dataset to only include the USD/EUR conversion rates and filter the data to include the 2021 data only. Finally, we can rename the column headers to make them more consistent with each other.

The data sources are now ready to be merged into a single DataFrame, there are a number of ways to do this using the `Pandas` package. We look at `concat()` and `merge()`, `concat()` works as appending tables whereas `merge()` works similar to joining in SQL. As we need to have data for both prices and exchange rates, inner join is the appropriate choice to merge the data with the `merge()` function. Looking at the joined DataFrame, we can see that there are still missing records in the data. By running `dropna()`, we now have a DataFrame that has no missing data. Finally, the prices for AAPL and BTC in EUR can be found by dividing the USD-EUR conversion rate to the US dollars prices.

Before data analysis, we can sort our data with `sort_values()` and `sort_index()`, we will sort the data by `Ticker` and then `Date` (our index). This table will be use in our plots and data analysis later in order to confirm the data is in the order we want. We will also create a new column `Month` to allow us to group the data. If required, the DataFrame can be extracted into Lists, which can be joined into a Dict, or be extracted into a 2D `Numpy` array. As you will see in the code, you can do this by using a for loop with `iterrows()`. The 2D `Numpy` array will allow us to apply mathematical functions to the data such as `round()` to round the prices, or to find the correlation between the 2 arrays by using `corrcoef()`. We can demonstrate the correlation between AAPL and BTC rounded to 5 decimal places in the code.

Alternatively, we can use `groupby()` and `agg()` to our DataFrame to see aggregations of the data seperated by groups desired. For this report, we will get the minimum, mean , maximum, and standard deviation for the prices for each instrument per month. We will also add 2 new columns containing the name of the instrument and the month of the data to simplify our visualisation process.

## Results

First, the correlation between the 2 prices is roughly 0.28264. A correlation close to 0 (and far away from -1 and 1) is deemed to be weak. So there is little correlation between prices of AAPL and BTC. In other words, we cannot simply predict/explain the price of AAPL or BTC given the other price.

Looking at the aggregated data shown below. It is clear that AAPL's prices are much lower than BTC generally. Scanning through the table, AAPL's highest price in 2021 was €160.16, whereas the lowest price for BTC was €25176.61.

Standard devisation is often used as a measure of risk and volatility of a finacial product. AAPL was the least volatile in March and the most volatile in November. Similarly, BTC was the least and most volatile in June and May, respectively. We can see more of this in our visualisations.

In [31]:
master_EUR_agg

Min          Mean           Max          STD  Ticker  \
Ticker Month                                                                  
AAPL   1        102.601673    108.355590    117.352591     4.863163    AAPL   
       2        100.393446    108.318874    113.776323     4.450951    AAPL   
       3         99.338425    101.845350    105.972755     1.580493    AAPL   
       4        104.502828    109.419984    111.957489     2.131128    AAPL   
       5        100.278709    103.994002    108.823702     2.439540    AAPL   
       6        101.406522    106.914632    113.931707     4.141694    AAPL   
       7        114.291494    121.623481    125.660270     3.121361    AAPL   
       8        121.987131    125.040625    128.460540     1.724993    AAPL   
       9        121.737522    126.091503    132.173914     3.306741    AAPL   
       10       119.725388    124.432103    128.691549     2.894300    AAPL   
       11       127.585077    134.118902    143.057227     5.633117    AAPL   
       12       139.611456    152.579424    160.156181     5.275295    AAPL   
BTCEUR 1      25176.611542  28541.751013  32299.519122  2252.214944  BTCEUR   
       2      27403.656178  37369.969103  47418.394693  5846.374951  BTCEUR   
       3      37467.438734  45570.316193  50260.364472  3483.738536  BTCEUR   
       4      40610.503077  47649.499280  53095.748835  3459.233847  BTCEUR   
       5      28414.971570  39193.787158  47868.248084  7100.530442  BTCEUR   
       6      26586.831136  30067.227599  33388.807374  1955.472334  BTCEUR   
       7      25311.149470  28887.244750  33686.055020  2372.364794  BTCEUR   
       8      32217.630918  38277.689661  42216.650474  3040.407520  BTCEUR   
       9      34681.518565  38775.922137  44401.754190  2391.195122  BTCEUR   
       10     37773.053610  49427.881755  56717.568424  4837.984655  BTCEUR   
       11     50008.486267  53796.751530  58348.220070  2556.523515  BTCEUR   
       12     41018.709607  44150.304540  50460.685328  2860.146245  BTCEUR   

                  Month  
Ticker Month             
AAPL   1        January  
       2       February  
       3          March  
       4          April  
       5            May  
       6           June  
       7           July  
       8         August  
       9      September  
       10       October  
       11      November  
       12      December  
BTCEUR 1        January  
       2       February  
       3          March  
       4          April  
       5            May  
       6           June  
       7           July  
       8         August  
       9      September  
       10       October  
       11      November  
       12      December

Plotting the mean price against the corresponding standard deviation, we can once again see that both price and standard deviation for BTC is generally much larger than AAPL. In fact, we can't see the AAPL clearly as the scales for BTC's data is just much bigger. This can be viewed as a visualisation of "High risk, high reward." Hovering the curser will show you the month each point represents.

In [42]:
fig1.show()

To look at the AAPL data, we can filter out the data by using `master_EUR_agg[master_EUR_agg['Ticker'] == "AAPL"]`.

In [43]:
fig2.show()

Now we can look at the line plot for the prices over the year. Again, the AAPL data is compressed to a line as the BTC prices are much larger and much more volatile. Hovering over the line will show you the price on a particular date. Recall that BTC was the most volatile in May and least in June, this can be seen as there is a large decrease in price for BTC in May and the price remained (relatively) stable afterwards in June.

In [44]:
fig3.show()

We can see the AAPL's line chart, the hugh spike in price in the last quarter of the year would explain the high standard deviation (volatility).

In [45]:
fig4.show()

We can also look at the mean price per month for both AAPL and BTC.

In [60]:
fig5.show()

In [61]:
fig6.show()

Finally, the candlestick plot below shows the daily AAPL prices in 2021 in US dollars. This shows each day's opening, closing, minimum and maximum prices. You can drag the bar at the bottom to narrow down the period of time shown.

In [62]:
fig7.show()

## Insights
(Point out at least 5 insights in bullet points)
- 
- 

## References

[Kaggle - Daily Exchange Rates per Euro 1999-2022](https://www.kaggle.com/datasets/lsind18/euro-exchange-daily-rates-19992020/)

[yfinance](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiz26v_p4P5AhWjMewKHSGaDBoQFnoECAQQAQ&url=https%3A%2F%2Fpypi.org%2Fproject%2Fyfinance%2F&usg=AOvVaw0_AxElbwZGER2NXe5JdGpa)